### Graph Class

In [1]:
from graph import Graph

# Example

In [2]:
G = Graph(weighted = True)

G.add_edge('a','b', 2)
G.add_edge('a','c', 4)
G.add_edge('b','c', 1)
G.add_edge('b','e', 5)
G.add_edge('b','d', 4)
G.add_edge('c','d', 2)
G.add_edge('c','f', 6)
G.add_edge('d','e', 1)
G.add_edge('d','f', 3)
G.add_edge('e','g', 4)
G.add_edge('f','g', 1)

graph = G.graph

### Single Source Shortest Path (Djikstra)

Only shortest paths distances

In [3]:
G.SSSP('c')

{'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6}

Printing the optimal paths


In [4]:
G.SSSP('c', get_paths = True)

({'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6},
 {'a': ['c', 'b', 'a'],
  'b': ['c', 'b'],
  'c': ['c'],
  'd': ['c', 'd'],
  'e': ['c', 'd', 'e'],
  'f': ['c', 'd', 'f'],
  'g': ['c', 'd', 'f', 'g']})

### (All-Pairs shortest path)

Floyd Warshall

In [5]:
G.APSP(algorithm = 'floyd warshall')

{'a': {'a': 0, 'b': 2, 'c': 3, 'd': 5, 'e': 6, 'f': 8, 'g': 9},
 'b': {'a': 2, 'b': 0, 'c': 1, 'd': 3, 'e': 4, 'f': 6, 'g': 7},
 'c': {'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6},
 'd': {'a': 5, 'b': 3, 'c': 2, 'd': 0, 'e': 1, 'f': 3, 'g': 4},
 'e': {'a': 6, 'b': 4, 'c': 3, 'd': 1, 'e': 0, 'f': 4, 'g': 4},
 'f': {'a': 8, 'b': 6, 'c': 5, 'd': 3, 'e': 4, 'f': 0, 'g': 1},
 'g': {'a': 9, 'b': 7, 'c': 6, 'd': 4, 'e': 4, 'f': 1, 'g': 0}}

Dijkstra

In [6]:
G.APSP(algorithm = 'dijkstra' )

{'a': {'a': 0, 'b': 2, 'c': 3, 'd': 5, 'e': 6, 'f': 8, 'g': 9},
 'b': {'a': 2, 'b': 0, 'c': 1, 'd': 3, 'e': 4, 'f': 6, 'g': 7},
 'c': {'a': 3, 'b': 1, 'c': 0, 'd': 2, 'e': 3, 'f': 5, 'g': 6},
 'd': {'a': 5, 'b': 3, 'c': 2, 'd': 0, 'e': 1, 'f': 3, 'g': 4},
 'e': {'a': 6, 'b': 4, 'c': 3, 'd': 1, 'e': 0, 'f': 4, 'g': 4},
 'f': {'a': 8, 'b': 6, 'c': 5, 'd': 3, 'e': 4, 'f': 0, 'g': 1},
 'g': {'a': 9, 'b': 7, 'c': 6, 'd': 4, 'e': 4, 'f': 1, 'g': 0}}

### Minimum spanning tree

Kruskal 

In [7]:
c,T = G.MST('kruskal')
c, T.tree

(10,
 {'a': {'b': 2},
  'b': {'a': 2, 'c': 1},
  'c': {'b': 1, 'd': 2},
  'd': {'c': 2, 'e': 1, 'f': 3},
  'e': {'d': 1},
  'f': {'d': 3, 'g': 1},
  'g': {'f': 1}})

Prim 

In [8]:
c,T = G.MST('prim', 'a')
c,T.tree

(10,
 {'a': {'b': 2},
  'b': {'a': 2, 'c': 1},
  'c': {'b': 1, 'd': 2},
  'd': {'c': 2, 'e': 1, 'f': 3},
  'e': {'d': 1},
  'f': {'d': 3, 'g': 1},
  'g': {'f': 1}})

# Example 2

In [9]:
G = Graph(weighted = True)

G.add_edge('a','b', 4)
G.add_edge('a','c', 6)
G.add_edge('a','d', 5)
G.add_edge('b','c', 1)
G.add_edge('c','d', 3)
G.add_edge('b','e', 7)
G.add_edge('c','e', 5)
G.add_edge('c','f', 4)
G.add_edge('d','f', 5)
G.add_edge('e','f', 1)
G.add_edge('e','g', 6)
G.add_edge('f','g', 8)

graph = G.graph

### Single Source Shortest Path (Djikstra)

Only shortest paths distances

In [10]:
G.SSSP('c')

{'a': 5, 'b': 1, 'c': 0, 'd': 3, 'e': 5, 'f': 4, 'g': 11}

Printing the optimal paths


In [11]:
G.SSSP('c', get_paths = True)

({'a': 5, 'b': 1, 'c': 0, 'd': 3, 'e': 5, 'f': 4, 'g': 11},
 {'a': ['c', 'b', 'a'],
  'b': ['c', 'b'],
  'c': ['c'],
  'd': ['c', 'd'],
  'e': ['c', 'e'],
  'f': ['c', 'f'],
  'g': ['c', 'e', 'g']})

### (All-Pairs shortest path)

Floyd Warshall

In [12]:
G.APSP(algorithm = 'floyd warshall')

{'a': {'a': 0, 'b': 4, 'c': 5, 'd': 5, 'e': 10, 'f': 9, 'g': 16},
 'b': {'a': 4, 'b': 0, 'c': 1, 'd': 4, 'e': 6, 'f': 5, 'g': 12},
 'c': {'a': 5, 'b': 1, 'c': 0, 'd': 3, 'e': 5, 'f': 4, 'g': 11},
 'd': {'a': 5, 'b': 4, 'c': 3, 'd': 0, 'e': 6, 'f': 5, 'g': 12},
 'e': {'a': 10, 'b': 6, 'c': 5, 'd': 6, 'e': 0, 'f': 1, 'g': 6},
 'f': {'a': 9, 'b': 5, 'c': 4, 'd': 5, 'e': 1, 'f': 0, 'g': 7},
 'g': {'a': 16, 'b': 12, 'c': 11, 'd': 12, 'e': 6, 'f': 7, 'g': 0}}

Dijkstra

In [13]:
G.APSP(algorithm = 'dijkstra' )

{'a': {'a': 0, 'b': 4, 'c': 5, 'd': 5, 'e': 10, 'f': 9, 'g': 16},
 'b': {'a': 4, 'b': 0, 'c': 1, 'd': 4, 'e': 6, 'f': 5, 'g': 12},
 'c': {'a': 5, 'b': 1, 'c': 0, 'd': 3, 'e': 5, 'f': 4, 'g': 11},
 'd': {'a': 5, 'b': 4, 'c': 3, 'd': 0, 'e': 6, 'f': 5, 'g': 12},
 'e': {'a': 10, 'b': 6, 'c': 5, 'd': 6, 'e': 0, 'f': 1, 'g': 6},
 'f': {'a': 9, 'b': 5, 'c': 4, 'd': 5, 'e': 1, 'f': 0, 'g': 7},
 'g': {'a': 16, 'b': 12, 'c': 11, 'd': 12, 'e': 6, 'f': 7, 'g': 0}}

### Minimum spanning tree

Kruskal 

In [14]:
c,T = G.MST('kruskal')
c, T.tree

(19,
 {'a': {'b': 4},
  'b': {'a': 4, 'c': 1},
  'c': {'b': 1, 'd': 3, 'f': 4},
  'd': {'c': 3},
  'e': {'f': 1, 'g': 6},
  'f': {'c': 4, 'e': 1},
  'g': {'e': 6}})

Prim 

In [15]:
c,T = G.MST('prim')
c,T.tree

(19,
 {'a': {'b': 4},
  'b': {'a': 4, 'c': 1},
  'c': {'b': 1, 'd': 3, 'f': 4},
  'd': {'c': 3},
  'e': {'f': 1, 'g': 6},
  'f': {'c': 4, 'e': 1},
  'g': {'e': 6}})